```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Presentation April 4th 2018, Reading, UK
This notebook contains animation and plots used for the 2nd GPU-Ocean presentation in the DARC seminars spring 2018.

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys
import time

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import GPUDrifterCollection
from SWESimulators import CPUDrifterCollection
from SWESimulators import DrifterEnsemble, WindForcingEnsemble
from SWESimulators import DataAssimilationUtils as dautils

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

# Simulator

In [ ]:
if 'gpuParticles' in globals():
    gpuParticles.cleanUp()

# Read some simulator field from netcfd:
netcdf_file_name = "netcdf_2018_02_12/CDKLM16_2018_02_12-16_52_49.out"
reload(CDKLM16)
if 'sim' in globals():
    sim.cleanUp()
sim = CDKLM16.CDKLM16.fromfilename(cl_ctx, \
                                   netcdf_file_name, \
                                   cont_write_netcdf=False)

#Calculate radius from center of bump for plotting
nx, ny, dx, dy = sim.nx, sim.ny, sim.dx, sim.dy
print (nx, ny)
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

validDomain = np.array([2,2,2,2])
eta1, hu1, hv1 = sim.download()
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
fig.suptitle("CDKLM simulator - Time = " + "{:04.0f}".format(0) + " s", fontsize=18)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

T = 100
sub_t = 10*sim.dt
def animate(i):
    if (i>0):
        t = sim.step(sub_t)
    else:
        t = 0.0

    eta1, hu1, hv1 = sim.download()
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
    fig.suptitle("CDKLM simulator - Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
  
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

# Simulator with passive drifters

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
if 'drifterEnsemble' in globals():
    drifterEnsemble.cleanUp()

# Read some simulator field from netcfd:
netcdf_file_name = "netcdf_2018_02_12/CDKLM16_2018_02_12-16_52_49.out"
sim = CDKLM16.CDKLM16.fromfilename(cl_ctx, \
                                   netcdf_file_name, \
                                   cont_write_netcdf=False)

#Calculate radius from center of bump for plotting
nx, ny, dx, dy = sim.nx, sim.ny, sim.dx, sim.dy
print (nx, ny)
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


validDomain = np.array([2,2,2,2])

## Define a bunch of particles to be released within the given domain
numParticles = 100
observation_variance = 5*sim.dx
drifterEnsemble = DrifterEnsemble.DrifterEnsemble(cl_ctx, numParticles,
                                             observation_variance=observation_variance)
drifterEnsemble.setGridInfoFromSim(sim)
drifterEnsemble.init()

eta1, hu1, hv1 = drifterEnsemble.sim.download()
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
plotter.showDrifters(drifterEnsemble.sim.drifters)


T = 100
sub_t = 1000*sim.dt
def animate(i):
    if (i>0):
        t = drifterEnsemble.step(sub_t)
    else:
        t = 0.0

    eta1, hu1, hv1 = drifterEnsemble.sim.download()
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
    plotter.showDrifters(drifterEnsemble.sim.drifters)
    
    fig.suptitle("Passive drifters - Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
  
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

# Data Assimilation on Drifter Ensemble

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
if 'drifterEnsemble' in globals():
    drifterEnsemble.cleanUp()
    
# Read some simulator field from netcfd:
netcdf_file_name = "netcdf_2018_02_12/CDKLM16_2018_02_12-16_52_49.out"
sim = CDKLM16.CDKLM16.fromfilename(cl_ctx, \
                                   netcdf_file_name, \
                                   cont_write_netcdf=False)
print (sim.nx, sim.ny)
# Init particles
reload(DrifterEnsemble)
np.random.seed(1)

numParticles = 50
observation_variance = 5*dx
resample_variance = 5*dx

## Define a bunch of particles to be released within the given domain
drifterEnsemble = DrifterEnsemble.DrifterEnsemble(cl_ctx, numParticles,
                                             observation_variance=observation_variance)
drifterEnsemble.setGridInfoFromSim(sim)
drifterEnsemble.init()

eta1, hu1, hv1 = drifterEnsemble.sim.download()
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

plotter.showDrifters(drifterEnsemble.sim.drifters)

T = 200
sub_t = 1000*sim.dt
drifterCollections = [drifterEnsemble.sim.drifters.copy()]
plotTitles = ["Initil ensemble"]

def animate(i):
    if (i>0):
        t = drifterEnsemble.step(sub_t)
    else:
        t = 0.0

    eta1, hu1, hv1 = drifterEnsemble.sim.download()
    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hu1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 hv1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    
    plotter.showDrifters(drifterEnsemble.sim.drifters)
     
    fig.suptitle("CDKLM16 with GPU drift and residualResampling - Time = " + "{:04.0f}".format(t) + " s", fontsize=18)
    
    if (i%50 == 0 and i > 0):
        drifterCollections.append(drifterEnsemble.sim.drifters.copy())
        plotTitles.append("Before particle filter at t = " + str(t))
        
        dautils.residualSampling(drifterEnsemble, reinitialization_variance=resample_variance)
         
        drifterCollections.append(drifterEnsemble.sim.drifters.copy())
        plotTitles.append("After particle filter at t = " + str(t))
    
    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax eta: " + str(np.max(eta1)) + \
        "\tMax hu: " + str(np.max(hu1)) + \
        "\tMax hv: " + str(np.max(hv1))
        print "\t\tObservation pos: ", drifterEnsemble.observeTrueState()
                     
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim



# Data Assimilation with changing model

In [ ]:
print "Started"
# Parameters
nx, ny, dx, dy = 50, 50, 5.0, 5.0
dt = 1
f = 0.0
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

try:
    if 'ensemble' in globals():
        ensemble.cleanUp()
except TypeError:
    pass
np.random.seed(2)
    
numParticles = 30
ensemble = WindForcingEnsemble.WindForcingEnsemble(numParticles, cl_ctx)
ensemble.setGridInfo(nx, ny, dx, dy, dt, boundaryConditions)
ensemble.setParameters(f)

driftersPerSim = 1
print "initializing ensemble with " + str(numParticles) + " particles (simulators)..."
ensemble.initWindCase(driftersPerSim)
print "ensemble of " + str(numParticles) + " ocean models, each with " + str(driftersPerSim) + " drifter(s)."
startPos = ensemble.observeParticles()
startObs = ensemble.observeTrueState()
#print "StartPos:\n", startPos
fig = plt.figure(figsize=(5,5))
plt.plot(startPos[:,0], startPos[:,1], 'mo', label='Init drifter position')
plt.plot(startObs[0], startObs[1], 'co', label='Init true state')
plt.ylim([0, ny*dy])
plt.xlim([0, nx*dx])

# initial model integration
#for i in range(10):
for i in range(5):
    print "iteration " + str(i)
    ensemble.step(100)
    pos = ensemble.observeParticles()
    obs = ensemble.observeTrueState()
    if i == 0:
        plt.plot(pos[:,0], pos[:,1], 'r.', label='drifter path')
        plt.plot(obs[0], obs[1], 'b.', label='true state path')
    else:
        plt.plot(pos[:,0], pos[:,1], 'r.')
        plt.plot(obs[0], obs[1], 'b.')
    #print pos
    


# Loop doing resampling + model integration
for pf in range(3):
    pos = ensemble.observeParticles()
    obs = ensemble.observeTrueState()
    if pf == 0:
        plt.plot(pos[:,0], pos[:,1], 'kx', label='observation particle')
    else:
        plt.plot(pos[:,0], pos[:,1], 'kx')
    plt.plot(obs[0], obs[1], 'kx')
    
    print "Resampling"
    dautils.probabilisticResampling(ensemble, reinitialization_variance=20)
    print "Resampling done"
    
    pos = ensemble.observeParticles()
    obs = ensemble.observeTrueState()
    if pf == 0:
        plt.plot(pos[:,0], pos[:,1], 'rs', label='resampled particles')
        plt.plot(obs[0], obs[1], 'bs', label='observation true state')
    else:
        plt.plot(pos[:,0], pos[:,1], 'rs')
        plt.plot(obs[0], obs[1], 'bs')
    
    #for i in range(10):
    for i in range(5):
        print "iteration " + str(i)
        ensemble.step(20)
        pos = ensemble.observeParticles()
        plt.plot(pos[:,0], pos[:,1], 'r.')
        obs = ensemble.observeTrueState()
        plt.plot(obs[0], obs[1], 'b.')
        #print pos

pos = ensemble.observeParticles()
plt.plot(pos[:,0], pos[:,1], 'rx', label='final observation particles')
obs = ensemble.observeTrueState()
plt.plot(obs[0], obs[1], 'bx', label='final observation true state')
plt.legend()

ensemble.plotDistanceInfo()

# Efficiency of drifter implementation
(if time)
Checking efficiency with {0, 1, 100, 1000, 10000} drifters.

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    del sim
if 'drifterEnsemble' in globals():
    drifterEnsemble.cleanUp()
    del drifterEnsemble
if 'ensemble' in globals():
    ensemble.cleanUp()
    del ensemble
        
times = np.zeros((10, 6))
numParticles = [0 , 1, 100, 10000, 100000, 1000000]
for j in range(10):
    for i in range(len(numParticles)):
        # Read some simulator field from netcfd:
        netcdf_file_name = "netcdf_2018_02_12/CDKLM16_2018_02_12-16_52_49.out"
        sim = CDKLM16.CDKLM16.fromfilename(cl_ctx, \
                                           netcdf_file_name, \
                                           cont_write_netcdf=False)
        if j == 0 and i == 0:
            print "(nx, ny)", (sim.nx, sim.ny)
            print "nx*ny = ", sim.nx*sim.ny
        nP = numParticles[i]
        if nP > 0:
            # add particles
            drifters = GPUDrifterCollection.GPUDrifterCollection(cl_ctx, nP,
                                                                 boundaryConditions=sim.boundary_conditions,
                                                                 domain_size_x=sim.dx*sim.nx,
                                                                 domain_size_y=sim.dy*sim.ny)
            drifters.initializeUniform()
            sim.attachDrifters(drifters)
            #print "Attached " + str(drifters.getNumDrifters()) + " drifters"

        #print "Start warm-up"
        warm_up_simt = sim.step(200*sim.dt)

        #print "Start experiment"
        start_time = time.time()
        t = sim.step(10000*sim.dt)
        stop_time = time.time()
        time_spent = stop_time - start_time
        print "Spent " + str(time_spent) + " seconds on " + str(nP) + " particles, run " + str(j)

        times[j,i] = time_spent
        sim.cleanUp()
    

averaged_times = np.mean(times, axis=0)
print averaged_times

normalizedT = np.array(averaged_times)
normalizedT = normalizedT/normalizedT[0]
fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, normalizedT, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '100', '10 000', '100 000', '1 000 000'))
        



In [ ]:
result = np.zeros(6)
result[0] = normalizedT[0]
result[1] = normalizedT[1]

fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, result, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '', '', '', ''))
plt.xlabel('Num drifters')
plt.ylabel('Rel. execution time')
plt.ylim(0, 1.8)

result[2] = normalizedT[2]
fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, result, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '100', '', '', ''))
plt.xlabel('Num drifters')
plt.ylabel('Rel. execution time')
plt.ylim(0, 1.8)


result[3] = normalizedT[3]
fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, result, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '100', '10 000', '', ''))
plt.xlabel('Num drifters')
plt.ylabel('Rel. execution time')
plt.ylim(0, 1.8)

result[4] = normalizedT[4]
fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, result, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '100', '10 000', '100 000', ''))
plt.xlabel('Num drifters')
plt.ylabel('Rel. execution time')
plt.ylim(0, 1.8)

result[5] = normalizedT[5]
fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, result, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '100', '10 000', '100 000', '1 000 000'))
plt.xlabel('Num drifters')
plt.ylabel('Rel. execution time')
plt.ylim(0, 1.8)


In [ ]:

if 'sim' in globals():
    sim.cleanUp()
    del sim
if 'drifterEnsemble' in globals():
    drifterEnsemble.cleanUp()
    del drifterEnsemble
if 'ensemble' in globals():
    ensemble.cleanUp()
    del ensemble
        
netcdf_file_name = "netcdf_2018_02_12/CDKLM16_2018_02_12-16_52_49.out"
sim = CDKLM16.CDKLM16.fromfilename(cl_ctx, \
                                   netcdf_file_name, \
                                   cont_write_netcdf=False)
eta, hu, hv = sim.download()
H0 = 10.0
nx, ny = sim.nx, sim.ny
dx, dy = sim.dx, sim.dy
dt = sim.dt

reload(CPUDrifterCollection)
times = np.zeros((10, 6))
#numParticles = [0 , 1, 100, 10000, 100000, 1000000]
numParticles = [0 , 1, 100, 100, 100, 100]

for j in range(10):
    for i in range(len(numParticles)):
        # Read some simulator field from netcfd:
        
        if j == 0 and i == 0:
            print "(nx, ny)", (sim.nx, sim.ny)
            print "nx*ny = ", sim.nx*sim.ny
        nP = numParticles[i]
        if nP == 0:
            times[j,i] = 0.0
            continue
        # add particles
        drifters = CPUDrifterCollection.CPUDrifterCollection(nP,
                                                             boundaryConditions=sim.boundary_conditions,
                                                             domain_size_x=sim.dx*sim.nx,
                                                             domain_size_y=sim.dy*sim.ny)
        drifters.initializeUniform()
        #print "Attached " + str(drifters.getNumDrifters()) + " drifters"

        #print "Start warm-up"
        for t in range(200):
            drifters.cpuDrift(eta, hu, hv, H0, nx, ny, dx, dy, dt)

        #print "Start experiment"
        start_time = time.time()
        for t in range(10000):
            drifters.cpuDrift(eta, hu, hv, H0, nx, ny, dx, dy, dt)
        stop_time = time.time()
        time_spent = stop_time - start_time
        print "Spent " + str(time_spent) + " seconds on " + str(nP) + " particles, run " + str(j)

        times[j,i] = time_spent
    

averaged_times = np.mean(times, axis=0)
print averaged_times

normalizedT = np.array(averaged_times)
normalizedT = normalizedT/normalizedT[0]
fig = plt.figure(figsize=(7,3))
x = np.arange(6)
plt.bar(x, normalizedT, align='center')
plt.grid()
plt.xticks(x, ('0', '1', '100', '10 000', '100 000', '1 000 000'))
        



In [ ]:
soar_x = np.linspace(0, 5)
soar_exp = (1 + soar_x/0.75)*np.exp(-soar_x/0.75)
fig = plt.figure(figsize=(4,2))
plt.plot(soar_x, soar_exp)
plt.ylabel("$Q/q_0$")
plt.xlabel("dist(a,b)")
plt.grid()


local_size = 16*16
cutoff_2_read = 20*20
cutoff_3_read = 22*22
print local_size, cutoff_2_read, cutoff_3_read
print cutoff_3_read*1.0/cutoff_2_read

cutoff_2_operations = 3*3
cutoff_3_operations = 5*5